<a href="https://colab.research.google.com/github/ShashankKamath/DecisionTree_ML/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
import numpy as np
from sklearn import svm

Saving hw2_question3.csv to hw2_question3.csv


In [3]:
data = pd.read_csv("hw2_question3.csv")
# header=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,50]
# header2=[31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46]
header=[1,3,5,6,7,8,10,13,15,16,17,18,19,21,24,27,29,30,31,32,33,34,35,36,37,39,41,42,44,46,50]
header2=[2,4,9,11,12,14,20,22,23,25,26,28,38,40,43,45]

data.columns=header
for i in header2:
    data.insert(i,i,0)

data.head()

,1,3,2,5,4,6,7,8,10,9,...,39,38,41,40,42,44,43,46,45,50
0,1,1,0,1,0,1,1,-1,0,0,...,0,0,-1,0,1,1,0,1,0,-1
1,1,0,0,1,0,1,1,-1,-1,0,...,1,0,-1,0,1,0,0,-1,0,-1
2,1,0,0,1,0,1,1,-1,-1,0,...,1,0,-1,0,1,-1,0,1,0,-1
3,1,0,0,-1,0,1,1,-1,1,0,...,0,0,-1,0,1,1,0,1,0,1
4,-1,0,0,-1,0,1,-1,-1,1,0,...,1,0,-1,0,1,-1,0,-1,0,1


In [0]:
f_columns=[3,10,13,21,24,27,39,44]
for index,row in data.iterrows():
  for c_no in f_columns:
    if row[c_no]==-1:
      row[c_no-1]=1
      row[c_no]=0
      row[c_no+1]=0
    elif row[c_no]==0:
      row[c_no-1]=0
      row[c_no]=1
      row[c_no+1]=0    
    elif row[c_no]==1:
      row[c_no-1]=0
      row[c_no]=0
      row[c_no+1]=1    

In [0]:
import random
dataset=data.values.tolist()
random.shuffle(dataset)
train_set=dataset[:7369]
test_set=dataset[7370:]

In [0]:
train_array=np.array(train_set)
train_array_X = train_array[:,:-1]
train_array_Y =train_array[:,-1]
test_array=np.array(test_set)
test_array_X=test_array[:,:-1]
test_array_Y=test_array[:,-1]

In [0]:
from random import seed
from random import randrange
def cv_split(data, folds=3):
	data_split = list()
	data_copy = list(data)
	fold_size = int(len(data) / folds)
	for i in range(folds):
		fld = list()
		while len(fld) < fold_size:
			index = randrange(len(data_copy))
			fld.append(data_copy.pop(index))
		data_split.append(fld)
	return data_split

In [0]:
import time
import operator
num_of_simulations=5
# c_list=[0.000001,0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]
c_list=list(np.arange(0.01,10,0.01))
total_accuracy=[]
total_time=[]

for c in range(len(c_list)):
  model = svm.SVC(C=c_list[c],kernel='linear')
#   print("C: ",c_list[c])
  total_accuracy.append(0)
  total_time.append(0)
  accuracy=[]
  running_time=[]
  for i in range(0,num_of_simulations):
#     print("Simulation No: ", i)
    seed(i)
    folds = cv_split(train_set, 3)
    for x in range(len(folds)):

        CVtest_set = folds[x]
        CVtrain_set = []
        for y in range(len(folds)):
          if y != x:
            CVtrain_set.extend(folds[y])

    CVtrain_array=np.array(CVtrain_set)
    CVtrain_array_X = CVtrain_array[:,:-1]
    CVtrain_array_Y =CVtrain_array[:,-1]

    CVtest_array=np.array(CVtest_set)
    CVtest_array_X=CVtest_array[:,:-1]
    CVtest_array_Y=CVtest_array[:,-1]
    
    start_time=time.time()
    model.fit(CVtrain_array_X,CVtrain_array_Y)
    stop_time=time.time()
    running_time.append(stop_time-start_time)
    
    class_predicted=[]
    class_predicted=model.predict(CVtest_array_X)
    score=0
    for j in range(0,len(CVtest_array_Y)):
      if CVtest_array_Y[j]==class_predicted[j]:
        score+=1
    accuracy.append(score/len(class_predicted))
    
#     print("Accuracy: ", accuracy)
  total_time[c]=sum(running_time)/len(running_time)
  total_accuracy[c]=(sum(accuracy)/len(accuracy))*100

Using the Best C on Test Data

In [0]:
max_index=[i for i, x in enumerate(total_accuracy) if x == max(total_accuracy)]
max_accuracy=0
for index in range(len(max_index)):
  c_best=c_list[max_index[index]]
  model = svm.SVC(C=c_best,kernel='linear')
  model.fit(train_array_X,train_array_Y)
  y_predict=model.predict(test_array_X)
  score=0
  for i in range(0,len(test_array_Y)):
    if test_array_Y[i]==y_predict[i]:
      score+=1
  accuracy=(score/len(y_predict))*100
  if accuracy>=max_accuracy:
    max_accuracy=accuracy
    c=c_best
print("Accuracy: ", max_accuracy)
print("C:", c)

for 5 sims
Accuracy:  94.00108577633007 
C: 0.87

for 1 sim
Accuracy:  94.10966340933767 
C: 1.6299999999999992

For Non-Linear Functions:


In [0]:
c_list=[0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.06999999999999999, 0.08, 0.09, 0.09999999999999999, 0.11, 0.12, 0.13, 0.14, 0.15000000000000002, 0.16, 0.17, 0.18000000000000002, 0.19, 0.2, 0.21000000000000002, 0.22, 0.23, 0.24000000000000002, 0.25, 0.26, 0.27, 0.28, 0.29000000000000004, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35000000000000003, 0.36000000000000004, 0.37, 0.38, 0.39, 0.4, 0.41000000000000003, 0.42000000000000004, 0.43, 0.44, 0.45, 0.46, 0.47000000000000003, 0.48000000000000004, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.5700000000000001, 0.5800000000000001, 0.59, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.66, 0.67, 0.68, 0.6900000000000001, 0.7000000000000001, 0.7100000000000001, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79, 0.8, 0.81, 0.8200000000000001, 0.8300000000000001, 0.8400000000000001, 0.85, 0.86, 0.87, 0.88, 0.89, 0.9, 0.91, 0.92, 0.93, 0.9400000000000001, 0.9500000000000001, 0.9600000000000001, 0.97, 0.98, 0.99, 1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1, 1.11, 1.12, 1.1300000000000001, 1.1400000000000001, 1.1500000000000001, 1.1600000000000001, 1.17, 1.18, 1.19, 1.2, 1.21, 1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3, 1.31, 1.32, 1.33, 1.34, 1.35, 1.36, 1.37, 1.3800000000000001, 1.3900000000000001, 1.4000000000000001, 1.4100000000000001, 1.42, 1.43, 1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 1.5, 1.51, 1.52, 1.53, 1.54, 1.55, 1.56, 1.57, 1.58, 1.59, 1.6, 1.61, 1.62, 1.6300000000000001, 1.6400000000000001, 1.6500000000000001, 1.6600000000000001, 1.6700000000000002, 1.68, 1.69, 1.7, 1.71, 1.72, 1.73, 1.74, 1.75, 1.76, 1.77, 1.78, 1.79, 1.8, 1.81, 1.82, 1.83, 1.84, 1.85, 1.86, 1.87, 1.8800000000000001, 1.8900000000000001, 1.9000000000000001, 1.9100000000000001, 1.9200000000000002, 1.93, 1.94, 1.95, 1.96, 1.97, 1.98, 1.99, 2.0, 2.01, 2.02, 2.03, 2.04, 2.05, 2.0599999999999996, 2.07, 2.0799999999999996, 2.09, 2.0999999999999996, 2.11, 2.1199999999999997, 2.13, 2.1399999999999997, 2.15, 2.1599999999999997, 2.17, 2.1799999999999997, 2.19, 2.1999999999999997, 2.21, 2.2199999999999998, 2.23, 2.2399999999999998, 2.25, 2.26, 2.27, 2.28, 2.29, 2.3, 2.31, 2.32, 2.3299999999999996, 2.34, 2.3499999999999996, 2.36, 2.3699999999999997, 2.38, 2.3899999999999997, 2.4, 2.4099999999999997, 2.42, 2.4299999999999997, 2.44, 2.4499999999999997, 2.46, 2.4699999999999998, 2.48, 2.4899999999999998, 2.5, 2.51, 2.52, 2.53, 2.54, 2.55, 2.56, 2.57, 2.5799999999999996, 2.59, 2.5999999999999996, 2.61, 2.6199999999999997, 2.63, 2.6399999999999997, 2.65, 2.6599999999999997, 2.67, 2.6799999999999997, 2.69, 2.6999999999999997, 2.71, 2.7199999999999998, 2.73, 2.7399999999999998, 2.75, 2.76, 2.77, 2.78, 2.79, 2.8, 2.81, 2.82, 2.8299999999999996, 2.84, 2.8499999999999996, 2.86, 2.8699999999999997, 2.88, 2.8899999999999997, 2.9, 2.9099999999999997, 2.92, 2.9299999999999997, 2.94, 2.9499999999999997, 2.96, 2.9699999999999998, 2.98, 2.9899999999999998, 3.0, 3.01, 3.02, 3.03, 3.04, 3.05, 3.06, 3.07, 3.08, 3.09, 3.0999999999999996, 3.11, 3.1199999999999997, 3.13, 3.1399999999999997, 3.15, 3.1599999999999997, 3.17, 3.1799999999999997, 3.19, 3.1999999999999997, 3.21, 3.2199999999999998, 3.23, 3.2399999999999998, 3.25, 3.26, 3.27, 3.28, 3.29, 3.3, 3.31, 3.32, 3.33, 3.34, 3.3499999999999996, 3.36, 3.3699999999999997, 3.38, 3.3899999999999997, 3.4, 3.4099999999999997, 3.42, 3.4299999999999997, 3.44, 3.4499999999999997, 3.46, 3.4699999999999998, 3.48, 3.4899999999999998, 3.5, 3.51, 3.52, 3.53, 3.54, 3.55, 3.56, 3.57, 3.58, 3.59, 3.5999999999999996, 3.61, 3.6199999999999997, 3.63, 3.6399999999999997, 3.65, 3.6599999999999997, 3.67, 3.6799999999999997, 3.69, 3.6999999999999997, 3.71, 3.7199999999999998, 3.73, 3.7399999999999998, 3.75, 3.76, 3.77, 3.78, 3.79, 3.8, 3.81, 3.82, 3.83, 3.84, 3.8499999999999996, 3.86, 3.8699999999999997, 3.88, 3.8899999999999997, 3.9, 3.9099999999999997, 3.92, 3.9299999999999997, 3.94, 3.9499999999999997, 3.96, 3.9699999999999998, 3.98, 3.9899999999999998, 4.0, 4.01, 4.02, 4.03, 4.04, 4.05, 4.06, 4.07, 4.08, 4.09, 4.1, 4.109999999999999, 4.12, 4.13, 4.14, 4.1499999999999995, 4.16, 4.17, 4.18, 4.1899999999999995, 4.2, 4.21, 4.22, 4.2299999999999995, 4.24, 4.25, 4.26, 4.27, 4.28, 4.29, 4.3, 4.31, 4.32, 4.33, 4.34, 4.35, 4.36, 4.37, 4.38, 4.39, 4.3999999999999995, 4.41, 4.42, 4.43, 4.4399999999999995, 4.45, 4.46, 4.47, 4.4799999999999995, 4.49, 4.5, 4.51, 4.52, 4.53, 4.54, 4.55, 4.56, 4.57, 4.58, 4.59, 4.6, 4.61, 4.62, 4.63, 4.64, 4.6499999999999995, 4.66, 4.67, 4.68, 4.6899999999999995, 4.7, 4.71, 4.72, 4.7299999999999995, 4.74, 4.75, 4.76, 4.77, 4.78, 4.79, 4.8, 4.81, 4.82, 4.83, 4.84, 4.85, 4.86, 4.87, 4.88, 4.89, 4.8999999999999995, 4.91, 4.92, 4.93, 4.9399999999999995, 4.95, 4.96, 4.97, 4.9799999999999995, 4.99, 5.0, 5.01, 5.02, 5.03, 5.04, 5.05, 5.06, 5.07, 5.08, 5.09, 5.1, 5.11, 5.12, 5.13, 5.14, 5.1499999999999995, 5.16, 5.17, 5.18, 5.1899999999999995, 5.2, 5.21, 5.22, 5.2299999999999995, 5.24, 5.25, 5.26, 5.27, 5.28, 5.29, 5.3, 5.31, 5.32, 5.33, 5.34, 5.35, 5.36, 5.37, 5.38, 5.39, 5.3999999999999995, 5.41, 5.42, 5.43, 5.4399999999999995, 5.45, 5.46, 5.47, 5.4799999999999995, 5.49, 5.5, 5.51, 5.52, 5.53, 5.54, 5.55, 5.56, 5.57, 5.58, 5.59, 5.6, 5.61, 5.62, 5.63, 5.64, 5.6499999999999995, 5.66, 5.67, 5.68, 5.6899999999999995, 5.7, 5.71, 5.72, 5.7299999999999995, 5.74, 5.75, 5.76, 5.77, 5.78, 5.79, 5.8, 5.81, 5.82, 5.83, 5.84, 5.85, 5.86, 5.87, 5.88, 5.89, 5.8999999999999995, 5.91, 5.92, 5.93, 5.9399999999999995, 5.95, 5.96, 5.97, 5.9799999999999995, 5.99, 6.0, 6.01, 6.02, 6.03, 6.04, 6.05, 6.06, 6.07, 6.08, 6.09, 6.1, 6.11, 6.12, 6.13, 6.14, 6.15, 6.16, 6.17, 6.18, 6.1899999999999995, 6.2, 6.21, 6.22, 6.2299999999999995, 6.24, 6.25, 6.26, 6.27, 6.28, 6.29, 6.3, 6.31, 6.32, 6.33, 6.34, 6.35, 6.36, 6.37, 6.38, 6.39, 6.4, 6.41, 6.42, 6.43, 6.4399999999999995, 6.45, 6.46, 6.47, 6.4799999999999995, 6.49, 6.5, 6.51, 6.52, 6.53, 6.54, 6.55, 6.56, 6.57, 6.58, 6.59, 6.6, 6.61, 6.62, 6.63, 6.64, 6.65, 6.66, 6.67, 6.68, 6.6899999999999995, 6.7, 6.71, 6.72, 6.7299999999999995, 6.74, 6.75, 6.76, 6.77, 6.78, 6.79, 6.8, 6.81, 6.82, 6.83, 6.84, 6.85, 6.86, 6.87, 6.88, 6.89, 6.9, 6.91, 6.92, 6.93, 6.9399999999999995, 6.95, 6.96, 6.97, 6.9799999999999995, 6.99, 7.0, 7.01, 7.02, 7.03, 7.04, 7.05, 7.06, 7.07, 7.08, 7.09, 7.1, 7.11, 7.12, 7.13, 7.14, 7.15, 7.16, 7.17, 7.18, 7.1899999999999995, 7.2, 7.21, 7.22, 7.2299999999999995, 7.24, 7.25, 7.26, 7.27, 7.28, 7.29, 7.3, 7.31, 7.32, 7.33, 7.34, 7.35, 7.36, 7.37, 7.38, 7.39, 7.4, 7.41, 7.42, 7.43, 7.4399999999999995, 7.45, 7.46, 7.47, 7.4799999999999995, 7.49, 7.5, 7.51, 7.52, 7.53, 7.54, 7.55, 7.56, 7.57, 7.58, 7.59, 7.6, 7.61, 7.62, 7.63, 7.64, 7.65, 7.66, 7.67, 7.68, 7.6899999999999995, 7.7, 7.71, 7.72, 7.7299999999999995, 7.74, 7.75, 7.76, 7.77, 7.78, 7.79, 7.8, 7.81, 7.82, 7.83, 7.84, 7.85, 7.86, 7.87, 7.88, 7.89, 7.9, 7.91, 7.92, 7.93, 7.94, 7.95, 7.96, 7.97, 7.9799999999999995, 7.99, 8.0, 8.01, 8.02, 8.03, 8.04, 8.05, 8.06, 8.07, 8.08, 8.09, 8.1, 8.11, 8.12, 8.13, 8.14, 8.15, 8.16, 8.17, 8.18, 8.19, 8.2, 8.209999999999999, 8.22, 8.23, 8.24, 8.25, 8.26, 8.27, 8.28, 8.29, 8.3, 8.31, 8.32, 8.33, 8.34, 8.35, 8.36, 8.37, 8.38, 8.39, 8.4, 8.41, 8.42, 8.43, 8.44, 8.45, 8.459999999999999, 8.47, 8.48, 8.49, 8.5, 8.51, 8.52, 8.53, 8.54, 8.55, 8.56, 8.57, 8.58, 8.59, 8.6, 8.61, 8.62, 8.63, 8.64, 8.65, 8.66, 8.67, 8.68, 8.69, 8.7, 8.71, 8.72, 8.73, 8.74, 8.75, 8.76, 8.77, 8.78, 8.79, 8.8, 8.81, 8.82, 8.83, 8.84, 8.85, 8.86, 8.87, 8.88, 8.89, 8.9, 8.91, 8.92, 8.93, 8.94, 8.95, 8.96, 8.97, 8.98, 8.99, 9.0, 9.01, 9.02, 9.03, 9.04, 9.05, 9.06, 9.07, 9.08, 9.09, 9.1, 9.11, 9.12, 9.13, 9.14, 9.15, 9.16, 9.17, 9.18, 9.19, 9.2, 9.21, 9.22, 9.23, 9.24, 9.25, 9.26, 9.27, 9.28, 9.29, 9.3, 9.31, 9.32, 9.33, 9.34, 9.35, 9.36, 9.37, 9.38, 9.39, 9.4, 9.41, 9.42, 9.43, 9.44, 9.45, 9.46, 9.47, 9.48, 9.49, 9.5, 9.51, 9.52, 9.53, 9.54, 9.55, 9.56, 9.57, 9.58, 9.59, 9.6, 9.61, 9.62, 9.63, 9.64, 9.65, 9.66, 9.67, 9.68, 9.69, 9.7, 9.71, 9.72, 9.73, 9.74, 9.75, 9.76, 9.77, 9.78, 9.79, 9.8, 9.81, 9.82, 9.83, 9.84, 9.85, 9.86, 9.87, 9.88, 9.89, 9.9, 9.91, 9.92, 9.93, 9.94, 9.95, 9.96, 9.97, 9.98, 9.99]
total_accuracy=[92.89902280130292, 93.28990228013029, 93.54234527687295, 93.67263843648207, 93.68078175895764, 93.67263843648207, 93.71335504885992, 93.75407166123779, 93.75407166123777, 93.77035830618891, 93.75407166123779, 93.76221498371335, 93.73778501628664, 93.74592833876221, 93.80293159609121, 93.80293159609121, 93.78664495114005, 93.77035830618891, 93.81107491856679, 93.83550488599349, 93.7785016286645, 93.81107491856679, 93.82736156351793, 93.83550488599349, 93.80293159609121, 93.79478827361562, 93.81921824104234, 93.80293159609121, 93.81921824104234, 93.81107491856676, 93.80293159609121, 93.83550488599347, 93.84364820846905, 93.78664495114005, 93.80293159609121, 93.78664495114008, 93.78664495114005, 93.80293159609121, 93.80293159609121, 93.78664495114005, 93.80293159609121, 93.81107491856679, 93.82736156351793, 93.81921824104234, 93.81107491856679, 93.81921824104236, 93.81107491856679, 93.81921824104236, 93.80293159609121, 93.81921824104234, 93.82736156351793, 93.79478827361565, 93.79478827361562, 93.81107491856679, 93.81921824104234, 93.81107491856676, 93.81107491856679, 93.79478827361562, 93.80293159609121, 93.81107491856679, 93.81107491856679, 93.81921824104234, 93.82736156351793, 93.81921824104234, 93.81921824104234, 93.83550488599347, 93.83550488599349, 93.82736156351793, 93.82736156351793, 93.82736156351793, 93.82736156351793, 93.81921824104234, 93.81921824104234, 93.83550488599347, 93.83550488599347, 93.84364820846905, 93.81921824104234, 93.81107491856679, 93.84364820846905, 93.83550488599349, 93.84364820846905, 93.83550488599349, 93.82736156351793, 93.82736156351793, 93.82736156351793, 93.83550488599349, 93.85179153094462, 93.83550488599347, 93.82736156351793, 93.83550488599347, 93.81921824104234, 93.83550488599347, 93.83550488599349, 93.83550488599347, 93.80293159609121, 93.82736156351793, 93.81921824104234, 93.81921824104236, 93.82736156351793, 93.81921824104234, 93.82736156351793, 93.81107491856679, 93.80293159609121, 93.80293159609121, 93.81107491856679, 93.77035830618891, 93.79478827361562, 93.81921824104234, 93.83550488599349, 93.81107491856679, 93.78664495114008, 93.81107491856679, 93.79478827361562, 93.81107491856679, 93.78664495114005, 93.78664495114005, 93.78664495114005, 93.80293159609121, 93.7785016286645, 93.78664495114005, 93.78664495114005, 93.76221498371336, 93.76221498371336, 93.75407166123779, 93.77035830618891, 93.75407166123779, 93.77035830618894, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.77035830618894, 93.76221498371336, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.77035830618891, 93.77035830618891, 93.76221498371335, 93.76221498371335, 93.76221498371335, 93.7785016286645, 93.77035830618891, 93.77035830618891, 93.7785016286645, 93.77035830618891, 93.7785016286645, 93.7785016286645, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.77035830618894, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.76221498371336, 93.76221498371336, 93.77035830618894, 93.7785016286645, 93.77035830618891, 93.76221498371336, 93.77035830618891, 93.76221498371336, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.77035830618891, 93.79478827361562, 93.78664495114008, 93.78664495114008, 93.7785016286645, 93.78664495114008, 93.7785016286645, 93.78664495114008, 93.7785016286645, 93.81107491856679, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.78664495114008, 93.78664495114008, 93.7785016286645, 93.77035830618891, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.78664495114008, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.79478827361562, 93.79478827361562, 93.79478827361562, 93.78664495114008, 93.7785016286645, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.7785016286645, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.77035830618891, 93.76221498371336, 93.77035830618891, 93.7785016286645, 93.77035830618891, 93.7785016286645, 93.7785016286645, 93.7785016286645, 93.79478827361562, 93.76221498371335, 93.77035830618891, 93.76221498371335, 93.77035830618891, 93.76221498371335, 93.77035830618891, 93.77035830618891, 93.76221498371335, 93.77035830618891, 93.76221498371335, 93.77035830618891, 93.77035830618891, 93.76221498371335, 93.76221498371335, 93.77035830618891, 93.76221498371335, 93.75407166123779, 93.76221498371335, 93.76221498371335, 93.75407166123779, 93.76221498371335, 93.76221498371335, 93.76221498371335, 93.76221498371335, 93.76221498371335, 93.76221498371335, 93.75407166123779, 93.76221498371335, 93.76221498371335, 93.76221498371335, 93.75407166123779, 93.75407166123779, 93.76221498371335, 93.76221498371335, 93.76221498371335, 93.76221498371335, 93.75407166123779, 93.74592833876221, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.76221498371336, 93.75407166123779, 93.76221498371336, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.75407166123779, 93.73778501628665, 93.76221498371336, 93.74592833876221, 93.74592833876221, 93.74592833876221, 93.74592833876221, 93.73778501628665, 93.74592833876221, 93.74592833876221, 93.73778501628664, 93.73778501628664, 93.72964169381109, 93.73778501628664, 93.73778501628664, 93.73778501628664, 93.73778501628664, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.71335504885995, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.73778501628664, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.73778501628664, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.73778501628664, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.71335504885995, 93.72964169381109, 93.72964169381109, 93.71335504885995, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.72964169381109, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.71335504885995, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.72964169381109, 93.7214983713355, 93.71335504885995, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.72964169381109, 93.71335504885995, 93.7214983713355, 93.72964169381109, 93.71335504885995, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.71335504885995, 93.71335504885995, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.71335504885995, 93.71335504885995, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.71335504885995, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885995, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.73778501628665, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.73778501628665, 93.73778501628665, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.73778501628665, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.73778501628665, 93.73778501628665, 93.7214983713355, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.7214983713355, 93.73778501628665, 93.73778501628665, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.73778501628665, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.71335504885992, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.71335504885992, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.73778501628665, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.73778501628665, 93.73778501628665, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.71335504885992, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.71335504885992, 93.72964169381109, 93.73778501628665, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.71335504885992, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.73778501628665, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.71335504885992, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.73778501628665, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.71335504885992, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.72964169381109, 93.71335504885992, 93.72964169381109, 93.72964169381109, 93.71335504885992, 93.72964169381109, 93.72964169381109, 93.71335504885992, 93.72964169381109, 93.72964169381109, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.72964169381109, 93.71335504885992, 93.71335504885992, 93.72964169381109, 93.71335504885992, 93.71335504885992, 93.72964169381109, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.72964169381109, 93.71335504885992, 93.72964169381109, 93.71335504885992, 93.71335504885992, 93.72964169381109, 93.72964169381109, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.72964169381109, 93.71335504885992, 93.7214983713355, 93.72964169381109, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.72964169381109, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.72964169381109, 93.71335504885992, 93.73778501628665, 93.72964169381109, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.70521172638438, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.72964169381109, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.71335504885992, 93.71335504885992, 93.71335504885992, 93.7214983713355, 93.7214983713355, 93.7214983713355, 93.7214983713355]
total_time=[0.28603229522705076, 0.2632493019104004, 0.2450568199157715, 0.24665565490722657, 0.22647151947021485, 0.23686442375183106, 0.25765271186828614, 0.23605990409851074, 0.25005159378051756, 0.2524539947509766, 0.23926019668579102, 0.21927332878112793, 0.21807422637939453, 0.22207632064819335, 0.22247066497802734, 0.2212738037109375, 0.2204677104949951, 0.22326622009277344, 0.22347621917724608, 0.22166314125061035, 0.22567410469055177, 0.22506837844848632, 0.22867069244384766, 0.22886037826538086, 0.2284687042236328, 0.22746939659118653, 0.22587432861328124, 0.23266229629516602, 0.23165493011474608, 0.239259672164917, 0.24666366577148438, 0.25004992485046384, 0.2392582893371582, 0.24186091423034667, 0.2436514377593994, 0.24186015129089355, 0.2574516773223877, 0.24486227035522462, 0.24725699424743652, 0.2542572021484375, 0.24805636405944825, 0.26764826774597167, 0.2514472007751465, 0.26684017181396485, 0.2582523822784424, 0.254243803024292, 0.26145172119140625, 0.27644124031066897, 0.26984372138977053, 0.2686450481414795, 0.2642490386962891, 0.2716397762298584, 0.273837947845459, 0.27944488525390626, 0.26944904327392577, 0.27544479370117186, 0.2734431266784668, 0.274840784072876, 0.2850342750549316, 0.28643035888671875, 0.28483266830444337, 0.27863922119140627, 0.2916294574737549, 0.2872336387634277, 0.30302796363830564, 0.310823917388916, 0.2994220733642578, 0.2976339817047119, 0.30202689170837405, 0.30342278480529783, 0.29602789878845215, 0.31482181549072263, 0.3084249973297119, 0.2988304615020752, 0.30602431297302246, 0.3030253410339355, 0.30402321815490724, 0.3024250030517578, 0.29982562065124513, 0.30802140235900877, 0.3018235683441162, 0.30661725997924805, 0.3068227767944336, 0.30422658920288087, 0.3108105182647705, 0.3362088203430176, 0.3166149616241455, 0.3092180728912354, 0.3214146137237549, 0.32361564636230467, 0.313216495513916, 0.31742143630981445, 0.3140255928039551, 0.3106250286102295, 0.3130175113677979, 0.31202025413513185, 0.30402522087097167, 0.31761975288391114, 0.32581171989440916, 0.3186202049255371, 0.31841750144958497, 0.3124251842498779, 0.3212145805358887, 0.3212155818939209, 0.3240070819854736, 0.3204094886779785, 0.3348092555999756, 0.32680621147155764, 0.3242093563079834, 0.32221231460571287, 0.3312049865722656, 0.33620829582214357, 0.3334078311920166, 0.33980493545532225, 0.34760055541992185, 0.34220333099365235, 0.33360700607299804, 0.3332045078277588, 0.33820366859436035, 0.34540615081787107, 0.3419989585876465, 0.34740438461303713, 0.34420194625854494, 0.33980755805969237, 0.3495967388153076, 0.34399943351745604, 0.34800262451171876, 0.3509974956512451, 0.3525979518890381, 0.34540019035339353, 0.3646282196044922, 0.35439410209655764, 0.3655890941619873, 0.3547957897186279, 0.35838890075683594, 0.3625938415527344, 0.38768930435180665, 0.3751856327056885, 0.3667890548706055, 0.37678256034851076, 0.35599532127380373, 0.35599818229675295, 0.3671885013580322, 0.3687872409820557, 0.3601893424987793, 0.3713962554931641, 0.36599278450012207, 0.37378406524658203, 0.38457584381103516, 0.36878552436828616, 0.37299251556396484, 0.37218427658081055, 0.36559128761291504, 0.3725916385650635, 0.3831819534301758, 0.3923784255981445, 0.3823796272277832, 0.38757848739624023, 0.3859753131866455, 0.38797893524169924, 0.41636033058166505, 0.41396369934082033, 0.38717103004455566, 0.4211552619934082, 0.397568941116333, 0.3931704521179199, 0.4025688648223877, 0.40516247749328616, 0.4117581367492676, 0.43154635429382326, 0.4209627151489258, 0.40716280937194826, 0.4415480613708496, 0.4225490093231201, 0.3953685760498047, 0.40337090492248534, 0.432354211807251, 0.43454627990722655, 0.43395304679870605, 0.42514853477478026, 0.4753251075744629, 0.4255502700805664, 0.4185596466064453, 0.41615819931030273, 0.46972246170043946, 0.4831266403198242, 0.5692769050598144, 0.4331113338470459, 0.4734452724456787, 0.4307541370391846, 0.48966155052185056, 0.4148214340209961, 0.40856518745422366, 0.42255096435546874, 0.4721231937408447, 0.4621315002441406, 0.48192882537841797, 0.4713315010070801, 0.4577305793762207, 0.4375442981719971, 0.4211526870727539, 0.4187479972839355, 0.4209620475769043, 0.4563325881958008, 0.4317472457885742, 0.423158073425293, 0.43175463676452636, 0.4439373970031738, 0.46833367347717286, 0.43874826431274416, 0.44155058860778806, 0.46653356552124026, 0.44634337425231935, 0.4583327293395996, 0.45073685646057127, 0.4429453372955322, 0.5009230136871338, 0.5444886207580566, 0.5234978199005127, 0.5039097785949707, 0.5408862113952637, 0.45593109130859377, 0.5171027660369873, 0.4717315673828125, 0.48811707496643064, 0.4995092868804932, 0.4685348987579346, 0.44893608093261717, 0.4617337226867676, 0.5263427257537842, 0.45813512802124023, 0.4821192264556885, 0.5386868476867676, 0.4971113681793213, 0.5304964065551758, 0.505507230758667, 0.5011034488677979, 0.4925191879272461, 0.5566801071166992, 0.4937137126922607, 0.4859182834625244, 0.5643153667449952, 0.545310115814209, 0.5438959121704101, 0.5101674556732178, 0.47998900413513185, 0.4884531021118164, 0.5021584987640381, 0.4900496482849121, 0.4930124282836914, 0.5247536182403565, 0.5156352996826172, 0.4929822444915771, 0.5025666236877442, 0.517003345489502, 0.5034884452819824, 0.53023681640625, 0.5535130500793457, 0.5259389877319336, 0.5508247375488281, 0.5380098819732666, 0.49806694984436034, 0.5589862823486328, 0.5070111751556396, 0.5513550758361816, 0.5444582939147949, 0.5726470947265625, 0.5374378204345703, 0.5477574348449707, 0.5518095016479492, 0.5707539081573486, 0.5750563621520997, 0.5349089622497558, 0.5634820938110352, 0.57788987159729, 0.5658619403839111, 0.5134412288665772, 0.6020917892456055, 0.6738006591796875, 0.5639142990112305, 0.5490242958068847, 0.566016435623169, 0.5550612926483154, 0.5715220928192138, 0.556787633895874, 0.5462336540222168, 0.5395869731903076, 0.5674526214599609, 0.5586077213287354, 0.5458592891693115, 0.6085572719573975, 0.560178565979004, 0.5424645900726318, 0.5612131118774414, 0.5447911262512207, 0.5482180118560791, 0.5431686878204346, 0.5636484622955322, 0.5427494049072266, 0.5394608497619628, 0.5386378288269043, 0.5542109489440918, 0.5675751209259033, 0.606068229675293, 0.6540040493011474, 0.5928558349609375, 0.5748566627502442, 0.5914430141448974, 0.5596884250640869, 0.5620652198791504, 0.553564977645874, 0.5519740581512451, 0.5771347999572753, 0.5727883338928222, 0.5466851711273193, 0.5542040824890136, 0.5480205535888671, 0.5577475547790527, 0.5429724216461181, 0.5735494136810303, 0.5753472805023193, 0.5766320705413819, 0.6099416732788085, 0.6234397411346435, 0.6286781311035157, 0.6154957771301269, 0.6111816883087158, 0.5781280994415283, 0.5895742416381836, 0.5771461963653565, 0.5740330219268799, 0.5861835956573487, 0.5697833061218261, 0.5701873779296875, 0.5690165519714355, 0.5855460166931152, 0.6403369426727294, 0.5934382438659668, 0.5602080821990967, 0.5849543571472168, 0.6041286945343017, 0.631671667098999, 0.6841325283050537, 0.6141067504882812, 0.5987429618835449, 0.5795750617980957, 0.5913702487945557, 0.5759564399719238, 0.5988668441772461, 0.6186814785003663, 0.6166844367980957, 0.6064142227172852, 0.5929077625274658, 0.6074761867523193, 0.6045102119445801, 0.5742552757263184, 0.6294915199279785, 0.6118110179901123, 0.616847038269043, 0.6706676483154297, 0.5945024490356445, 0.6128642082214355, 0.6960773944854737, 0.6343506813049317, 0.6009969234466552, 0.6058970928192139, 0.6119330883026123, 0.586650276184082, 0.6145915985107422, 0.6151356220245361, 0.6138523101806641, 0.6247043609619141, 0.6581280708312989, 0.6348968982696533, 0.6153569221496582, 0.6114643573760986, 0.6430583477020264, 0.6333287715911865, 0.6282607555389405, 0.6437100887298584, 0.6270819664001465, 0.603858757019043, 0.6432238101959229, 0.6262396335601806, 0.7243764877319336, 0.6042512416839599, 0.6808058261871338, 0.7509085178375244, 0.7381983757019043, 0.6648754119873047, 0.6663938522338867, 0.630421495437622, 0.6237749576568603, 0.6854311466217041, 0.6916241645812988, 0.6958512306213379, 0.7189235687255859, 0.7214329242706299, 0.7556747913360595, 0.7253360748291016, 0.6881205558776855, 0.7728127479553223, 0.6632481575012207, 0.712047004699707, 0.6728082656860351, 0.6768449306488037, 0.6604158401489257, 0.618636178970337, 0.6336296081542969, 0.602663230895996, 0.6288496017456054, 0.6088423252105712, 0.5952729701995849, 0.6542276859283447, 0.5980527400970459, 0.6256424903869628, 0.5938573837280273, 0.6014593601226806, 0.6232378959655762, 0.5976517200469971, 0.6028531074523926, 0.6192486763000489, 0.6222404956817627, 0.6452294826507569, 0.6171955108642578, 0.6598207473754882, 0.666614580154419, 0.6136442661285401, 0.6494206428527832, 0.6246418476104736, 0.617848253250122, 0.6252392768859864, 0.6156381130218506, 0.6086491584777832, 0.6340324401855468, 0.6222426891326904, 0.6396329402923584, 0.6322290420532226, 0.6576151847839355, 0.626836109161377, 0.6222399711608887, 0.6256396293640136, 0.6600224494934082, 0.6540267467498779, 0.6296345233917237, 0.6534211158752441, 0.656013822555542, 0.6428256034851074, 0.6318373680114746, 0.651621675491333, 0.6322374820709229, 0.6314353942871094, 0.6791105747222901, 0.6538220882415772, 0.6888052940368652, 0.6794081211090088, 0.7221781253814697, 0.6144466400146484, 0.7013945579528809, 0.7015990257263184, 0.6038545131683349, 0.6216403007507324, 0.6032471179962158, 0.6196447372436523, 0.6498241424560547, 0.6552162647247315, 0.6296407222747803, 0.6284343242645264, 0.6802064895629882, 0.6570194244384766, 0.6444300651550293, 0.6544258117675781, 0.6776090145111084, 0.6582137107849121, 0.6204428672790527, 0.6272408485412597, 0.657020378112793, 0.6436254978179932, 0.6414323806762695, 0.6682137012481689, 0.7141851425170899, 0.6510216236114502, 0.6576138973236084, 0.6304383277893066, 0.6200388908386231, 0.6346370697021484, 0.6462269783020019, 0.638630723953247, 0.6348313808441162, 0.6502280712127686, 0.6188474178314209, 0.6294379234313965, 0.6570261478424072, 0.6910033226013184, 0.6600168704986572, 0.6476239204406739, 0.6828098773956299, 0.6588298320770264, 0.6604204654693604, 0.6866013050079346, 0.6528203964233399, 0.6508275985717773, 0.6534216403961182, 0.7075939178466797, 0.706190538406372, 0.6840037345886231, 0.7395637512207032, 0.7070078372955322, 0.7102682590484619, 0.7014524459838867, 0.6908420562744141, 0.6868509292602539, 0.7019278526306152, 0.6788661003112793, 0.6884811878204345, 0.7334784984588623, 0.7162193775177002, 0.6829719543457031, 0.6791323184967041, 0.6935997009277344, 0.7366280078887939, 0.701030969619751, 0.7275445461273193, 0.7104727745056152, 0.689238166809082, 0.7311788558959961, 0.7032029151916503, 0.720273494720459, 0.7084098339080811, 0.6977969169616699, 0.7972079277038574, 0.7433500766754151, 0.7434628009796143, 0.7173601150512695, 0.765480899810791, 0.724855375289917, 0.7200115203857422, 0.7431835651397705, 0.7759347915649414, 0.7250170230865478, 0.7008630752563476, 0.6856284618377686, 0.6999188423156738, 0.7092114448547363, 0.7134419441223144, 0.7167510986328125, 0.7364066123962403, 0.7267699718475342, 0.7130691051483155, 0.7307345867156982, 0.7216572284698486, 0.7694922924041748, 0.7488009929656982, 0.7805325508117675, 0.7498328685760498, 0.7662150382995605, 0.7200341701507569, 0.7442965030670166, 0.7548299789428711, 0.7203033447265625, 0.7494402408599854, 0.7360134124755859, 0.7341712951660156, 0.7557807445526123, 0.7619442939758301, 0.7356131076812744, 0.7216524124145508, 0.7859434127807617, 0.7335552215576172, 0.7566855907440185, 0.8099730491638184, 0.7258073806762695, 0.7451472282409668, 0.7185905456542969, 0.782752275466919, 0.7985422611236572, 0.7840247631072998, 0.7716659545898438, 0.7464509010314941, 0.7599289894104004, 0.753653621673584, 0.7077253341674805, 0.7299859523773193, 0.7221875667572022, 0.7806572914123535, 0.7462236881256104, 0.7752179622650146, 0.758981466293335, 0.7432119369506835, 0.7753117084503174, 0.7484342098236084, 0.7859809875488282, 0.7661944389343261, 0.7411751747131348, 0.7499588966369629, 0.7439692497253418, 0.7379729270935058, 0.7387747287750244, 0.720383882522583, 0.7333806991577149, 0.7447661876678466, 0.7667512893676758, 0.7547678470611572, 0.8131299018859863, 0.7848805904388427, 0.7707220077514648, 0.8004551410675049, 0.7637928485870361, 0.7894413471221924, 0.7597020149230957, 0.763360071182251, 0.7503681659698487, 0.7501671791076661, 0.8206721782684326, 0.8366899013519287, 0.7983839988708497, 0.8107382774353027, 0.7712143421173095, 0.7938467025756836, 0.837472677230835, 0.7916230201721192, 0.7747303485870362, 0.7877449989318848, 0.8087002754211425, 0.7333311080932617, 0.7925681591033935, 0.7697886466979981, 0.7869231224060058, 0.7929761409759521, 0.7744136333465577, 0.7751519203186035, 0.8038164138793945, 0.7904281616210938, 0.7969006061553955, 0.7807538986206055, 0.7833832740783692, 0.8153292179107666, 0.8089459896087646, 0.7677595138549804, 0.772755241394043, 0.7523674488067627, 0.7501673698425293, 0.8647029399871826, 0.7717507362365723, 0.7645571231842041, 0.7929492473602295, 0.7631576061248779, 0.7623571872711181, 0.7683567523956298, 0.8060612678527832, 0.794736671447754, 0.7807505130767822, 0.8373718738555909, 0.8322858333587646, 0.7934313297271729, 0.781512451171875, 0.8094192028045655, 0.7988330364227295, 0.7973929882049561, 0.8321269989013672, 0.7959788322448731, 0.8022791385650635, 0.7781042098999024, 0.7946227550506592, 0.7902691841125489, 0.7686755180358886, 0.7806108474731446, 0.7941792488098145, 0.8119295120239258, 0.8225270748138428, 0.7811505794525146, 0.7998589038848877, 0.8182603359222412, 0.8523131370544433, 0.8255809307098388, 0.8749791622161865, 0.7865664005279541, 0.794392728805542, 0.7860215187072754, 0.8030050754547119, 0.8015697479248047, 0.8229843139648437, 0.8335317611694336, 0.8172164916992187, 0.8474378108978271, 0.8259086608886719, 0.8823050498962403, 0.8038071155548095, 0.8469164848327637, 0.8061753749847412, 0.8522781372070313, 0.8609994888305664, 0.8816396713256835, 0.9098910808563232, 0.8165935516357422, 0.8416836738586426, 0.8110341548919677, 0.8346925258636475, 0.8489934921264648, 0.8376424789428711, 0.8193923473358155, 0.8131341934204102, 0.8720337390899658, 0.8380898475646973, 0.8446386337280274, 0.8267498016357422, 0.9047266960144043, 0.8366163730621338, 0.8334338188171386, 0.8444188117980957, 0.9278930187225342, 0.830272626876831, 0.8502645015716552, 0.8702886581420899, 0.8478317737579346, 0.8845808029174804, 0.8643259525299072, 0.8192116737365722, 0.8203204154968262, 0.8969906330108642, 0.8485694885253906, 0.8623439788818359, 0.9233014106750488, 0.8199648380279541, 0.8999501705169678, 0.896324872970581, 0.8613501071929932, 0.8439486980438232, 0.8249235153198242, 0.8195231914520263, 0.8934919357299804, 0.8319156646728516, 0.8541096210479736, 0.853705883026123, 0.8487147808074951, 0.872294807434082, 0.8343240737915039, 0.8541056156158447, 0.8274454116821289, 0.9298666000366211, 0.9047159671783447, 0.8965360641479492, 0.8965157985687255, 0.856556510925293, 0.8398063659667969, 0.8704667091369629, 0.9332676410675049, 0.9067230701446534, 0.8668027877807617, 0.8341146945953369, 0.8655023574829102, 0.861898136138916, 0.8517079830169678, 0.9000808238983155, 0.8758876323699951, 0.9048822402954102, 0.9576131343841553, 0.8874852657318115, 0.8828870296478272, 0.8980493545532227, 0.8785295486450195, 0.9194921016693115, 0.9407260417938232, 0.8917896270751953, 0.8798468112945557, 0.8900524139404297, 0.9209360122680664, 0.9225832462310791, 0.932205057144165, 0.8886874198913575, 0.9058269023895263, 0.9226626396179199, 1.0128517150878906, 0.91898193359375, 0.9012284278869629, 0.898505687713623, 0.9365540027618409, 0.9175537586212158, 0.9261524677276611, 0.90605149269104, 0.938106107711792, 0.9373745441436767, 0.9916865825653076, 0.964511775970459, 0.8679078102111817, 0.9000791072845459, 0.8942876338958741, 0.8503052711486816, 0.8721045970916748, 0.8878865718841553, 0.9430478096008301, 0.8908819198608399, 0.9346628665924073, 0.87349534034729, 0.9448469638824463, 0.9483808994293212, 0.9227120876312256, 0.9610060691833496, 0.9554497718811035, 0.9377040863037109, 0.9299739837646485, 0.8732921600341796, 0.916471004486084, 0.9056096553802491, 0.9046246528625488, 0.8916364192962647, 0.9279048442840576, 0.921220064163208, 0.9685303688049316, 0.9184309482574463, 0.9337770938873291, 0.9080262184143066, 0.9526125907897949, 0.978919506072998, 0.9470437049865723, 0.880354356765747, 0.9307171821594238, 0.9096720218658447, 0.9120747566223144, 0.9276650428771973, 0.9114799976348877, 0.9970255374908448, 0.931671142578125, 0.9508495807647706, 0.9783452987670899, 0.9372361183166504, 0.923957633972168, 0.9208388328552246, 0.9913779735565186, 0.9118747711181641, 0.94774489402771, 0.9060592651367188, 0.9296717166900634, 0.9487802505493164, 0.9522491455078125, 0.9984380722045898, 0.9425106048583984, 0.9114217281341552, 0.9681684494018554, 0.9746829032897949, 0.9411614894866943, 0.9418377876281738, 0.9574527263641357, 0.9464532375335694, 0.9506515979766845, 0.9726385116577149, 0.9498539924621582, 1.0127714157104493, 0.9278728008270264, 0.9480912685394287, 0.947183895111084, 0.9647815704345704, 0.9527833938598633, 1.0199855327606202, 0.9707117557525635, 0.953454065322876, 0.9736364841461181, 0.9871338367462158, 0.9273697376251221, 0.9630450248718262, 0.9152713775634765, 0.9524545669555664, 0.9038761138916016, 0.8840935707092286, 0.9270648002624512, 0.9338594913482666, 0.9138725280761719, 0.9070799827575684, 0.9502524852752685, 0.9444529056549072, 0.9004817008972168, 0.9314587593078614, 0.9366538524627686, 0.9194732189178467, 0.9526511192321777, 0.9746381759643554, 0.9454526901245117, 0.8866820812225342, 0.928470230102539, 0.9244631290435791, 0.9416612148284912, 0.9766379833221436, 0.9310542106628418, 0.9238701820373535, 0.9154732704162598, 0.9004789829254151, 0.9486527442932129, 0.8792938232421875, 0.9582480907440185, 0.9110741138458252, 0.9062711715698242, 0.9490554332733154, 0.9546441078186035, 0.8958810329437256, 0.9612483501434326, 0.9244616985321045, 0.9068818092346191, 0.894084882736206, 0.9852288722991943, 0.9626450538635254, 0.9578476905822754, 0.9384544372558594, 0.9472506046295166, 0.9278621196746826, 0.9594406127929688, 0.9180683135986328, 0.9482522487640381, 0.9232697010040283, 0.9668445110321044, 0.9516486167907715, 0.9580455303192139, 0.925266170501709, 0.990021800994873, 0.9446604728698731, 0.9532505989074707, 0.9372608184814453, 0.9700441360473633, 0.9504535675048829, 0.9708354473114014, 0.9468555927276612, 0.9702401161193848, 0.950650405883789, 0.9456544399261475, 0.9440489768981933, 0.9620436668395996, 0.9644443988800049, 0.9550466060638427, 0.9558464050292969, 1.0188157558441162, 0.9354612827301025, 0.9576467037200928, 0.9548549652099609, 0.9120723247528076, 0.9710393905639648, 0.9808360576629639, 0.954651165008545, 1.010219192504883, 1.0006227016448974, 0.9608457088470459, 1.0332013607025146, 1.0837747573852539, 0.963043212890625, 1.0166117668151855, 0.9904261112213135, 0.9456608295440674, 0.9228667736053466, 0.9624422550201416, 1.0006231784820556, 0.9756397724151611, 0.9186679363250733, 0.9670433044433594, 0.950450086593628, 0.9518518924713135, 0.9558412075042725, 0.9602460384368896, 0.9776342868804931, 0.9856328010559082, 0.96764235496521, 0.973232650756836, 0.9334664821624756, 0.936460018157959, 0.9820367336273194, 0.9592440605163575, 0.9616459846496582, 0.9794298648834229, 1.0270106792449951, 1.0198095798492433, 0.9778363704681396, 1.0431936740875245, 0.9902270793914795, 0.9638377666473389, 1.0002232551574708, 0.9474534034729004, 0.9858304500579834, 0.9940271377563477, 0.9820361137390137, 1.0262065410614014, 1.0340039253234863, 1.0132155895233155, 0.9810310363769531, 1.033602523803711, 0.9818320751190186, 1.0927666187286378, 1.0238032341003418, 1.0206117630004883, 1.0030189037322998, 1.0010231494903565, 0.992222785949707, 0.9280643939971924, 0.9734382152557373, 1.0238095760345458, 0.9838328838348389, 0.9770360946655273, 1.0052130222320557, 1.0535878181457519, 1.0042223930358887, 1.004026508331299, 1.042595624923706, 1.0406052589416503]

In [0]:
import matplotlib.pyplot as plt
plt.plot(c_list,total_accuracy)
plt.xlabel("Misclassification Cost: C [0.01:10:0.01]")
plt.ylabel("Accuracy")
plt.title("Accuracy Vs Misclassification Cost")
plt.show()

In [0]:
plt.scatter(c_list,total_time)
plt.xlabel("Misclassification Cost: C [0.01:10:0.01]")
plt.ylabel("Average Time Taken at each C (sec)")
plt.title("Average Training Time Vs Misclassification Cost")
plt.show()

Quesiton Part 2
Using Non-Linear Kernels

In [0]:
import time
num_of_simulations=1
c_list=list(np.arange(0.01,100,0.1))
gamma_list=list(np.arange(0.0001,1,0.1))
total_accuracy=[]
total_time=[]

for c in range(len(c_list)):
    total_accuracy.append([])
    total_time.append([])
    for g in range(len(gamma_list)):
      model = svm.SVC(C=c_list[c],gamma=gamma_list[g],kernel='rbf')
    #   print("C: ",c_list[c])

      accuracy=[]
      running_time=[]
      for i in range(0,num_of_simulations):
    #     print("Simulation No: ", i)
        seed(i)
        folds = cv_split(train_set, 3)
        for x in range(len(folds)):

            CVtest_set = folds[x]
            CVtrain_set = []
            for y in range(len(folds)):
              if y != x:
                CVtrain_set.extend(folds[y])

        CVtrain_array=np.array(CVtrain_set)
        CVtrain_array_X = CVtrain_array[:,:-1]
        CVtrain_array_Y =CVtrain_array[:,-1]

        CVtest_array=np.array(CVtest_set)
        CVtest_array_X=CVtest_array[:,:-1]
        CVtest_array_Y=CVtest_array[:,-1]

        start_time=time.time()
        model.fit(CVtrain_array_X,CVtrain_array_Y)
        stop_time=time.time()
        running_time.append(stop_time-start_time)

        class_predicted=[]
        class_predicted=model.predict(CVtest_array_X)
        score=0
        for j in range(0,len(CVtest_array_Y)):
          if CVtest_array_Y[j]==class_predicted[j]:
            score+=1
        accuracy.append(score/len(class_predicted))

    #     print("Accuracy: ", accuracy)
      total_time[c].append(sum(running_time)/len(running_time))
      total_accuracy[c].append((sum(accuracy)/len(accuracy))*100)

total_accuracy=np.array(total_accuracy)
total_time=np.array(total_time)
max_index = np.where(total_accuracy == np.amax(total_accuracy))

max_accuracy=0
for index in range(len(max_index[0])):
  c_best=c_list[max_index[0][index]]
  g_best=gamma_list[max_index[1][index]]
  model = svm.SVC(C=c_best,gamma=g_best,kernel='rbf')
  model.fit(train_array_X,train_array_Y)
  y_predict=model.predict(test_array_X)
  score=0
  for i in range(0,len(test_array_Y)):
    if test_array_Y[i]==y_predict[i]:
      score+=1
  accuracy=(score/len(y_predict))*100
  if accuracy>=max_accuracy:
    max_accuracy=accuracy
    c=c_best
    g=g_best
print("Accuracy: ", max_accuracy)
print("C:", c)
print("Gamma: ",g)

Accuracy:  97.04125950054289
C: 1.2100000000000002
Gamma: 0.10010000000000001

In [0]:
import time
num_of_simulations=1
c_list=list(np.arange(0.01,100,0.1))
degree=[1,2,3,4,5]
gamma_list=list(np.arange(0.0001,1,0.1))
total_accuracy=[]
total_time=[]

for c in range(len(c_list)):
    total_accuracy.append([])
    total_time.append([])
    for g in range(len(degree)):
      model = svm.SVC(C=c_list[c],degree=degree[g],gamma=0.1,kernel='poly')
    #   print("C: ",c_list[c])

      accuracy=[]
      running_time=[]
      for i in range(0,num_of_simulations):
    #     print("Simulation No: ", i)
        seed(i)
        folds = cv_split(train_set, 3)
        for x in range(len(folds)):

            CVtest_set = folds[x]
            CVtrain_set = []
            for y in range(len(folds)):
              if y != x:
                CVtrain_set.extend(folds[y])

        CVtrain_array=np.array(CVtrain_set)
        CVtrain_array_X = CVtrain_array[:,:-1]
        CVtrain_array_Y =CVtrain_array[:,-1]

        CVtest_array=np.array(CVtest_set)
        CVtest_array_X=CVtest_array[:,:-1]
        CVtest_array_Y=CVtest_array[:,-1]

        start_time=time.time()
        model.fit(CVtrain_array_X,CVtrain_array_Y)
        stop_time=time.time()
        running_time.append(stop_time-start_time)

        class_predicted=[]
        class_predicted=model.predict(CVtest_array_X)
        score=0
        for j in range(0,len(CVtest_array_Y)):
          if CVtest_array_Y[j]==class_predicted[j]:
            score+=1
        accuracy.append(score/len(class_predicted))

    #     print("Accuracy: ", accuracy)
      total_time[c].append(sum(running_time)/len(running_time))
      total_accuracy[c].append((sum(accuracy)/len(accuracy))*100)

total_accuracy=np.array(total_accuracy)
total_time=np.array(total_time)
max_index = np.where(total_accuracy == np.amax(total_accuracy))

max_accuracy=0
for index in range(len(max_index[0])):
  c_best=c_list[max_index[0][index]]
  d_best=degree[max_index[1][index]]
  model = svm.SVC(C=c_best,degree=d_best,gamma=0.1,kernel='poly')
  model.fit(train_array_X,train_array_Y)
  y_predict=model.predict(test_array_X)
  score=0
  for i in range(0,len(test_array_Y)):
    if test_array_Y[i]==y_predict[i]:
      score+=1
  accuracy=(score/len(y_predict))*100
  if accuracy>=max_accuracy:
    max_accuracy=accuracy
    c=c_best
    d=d_best
print("Accuracy: ", max_accuracy)
print("C:", c)
print("Degree: ",d)

Poly
Accuracy:  96.66123778501628
C: 0.01
G=0.1
Degree:  5


Accuracy:  95.98262757871878
C: 0.01
Gamma: 1
Degree:  2

{'C': 0.1, 'degree': 3, 'gamma': 0.1, 'kernel': 'poly'}
Accuracy: 96.30836047774159

In [9]:
import time
num_of_simulations=1
c_list=[1.27]
gamma_list=[0.1]
##GAMMA after 1 waste
#c_list=list(np.arange(0.01,100,0.1))
#
#gamma_list=list(np.arange(0.0001,1,0.1))
#c_list=[0.1,1]
#gamma_list=[0.1]
degree=[5]
total_accuracy=[]
total_time=[]

for c in range(len(c_list)):
    total_accuracy.append([])
    total_time.append([])
    for g in range(len(gamma_list)):
        total_accuracy[c].append([])
        total_time[c].append([])
        for d in range(len(degree)):
          model = svm.SVC(C=c_list[c],degree=degree[d],gamma=gamma_list[g],kernel='poly')
        #   print("C: ",c_list[c])

          accuracy=[]
          running_time=[]
          for i in range(0,num_of_simulations):
        #     print("Simulation No: ", i)
            seed(i)
            folds = cv_split(train_set, 3)
            for x in range(len(folds)):

                CVtest_set = folds[x]
                CVtrain_set = []
                for y in range(len(folds)):
                  if y != x:
                    CVtrain_set.extend(folds[y])

            CVtrain_array=np.array(CVtrain_set)
            CVtrain_array_X = CVtrain_array[:,:-1]
            CVtrain_array_Y =CVtrain_array[:,-1]

            CVtest_array=np.array(CVtest_set)
            CVtest_array_X=CVtest_array[:,:-1]
            CVtest_array_Y=CVtest_array[:,-1]

            start_time=time.time()
            model.fit(CVtrain_array_X,CVtrain_array_Y)
            stop_time=time.time()
            running_time.append(stop_time-start_time)

            class_predicted=[]
            class_predicted=model.predict(CVtest_array_X)
            score=0
            for j in range(0,len(CVtest_array_Y)):
              if CVtest_array_Y[j]==class_predicted[j]:
                score+=1
            accuracy.append(score/len(class_predicted))

        #     print("Accuracy: ", accuracy)
          total_time[c][g].append(sum(running_time)/len(running_time))
          total_accuracy[c][g].append((sum(accuracy)/len(accuracy))*100)

total_accuracy=np.array(total_accuracy)
total_time=np.array(total_time)
ind = np.unravel_index(np.argmax(total_accuracy, axis=None), total_accuracy.shape)
model = svm.SVC(C=c_list[ind[0]],degree=degree[ind[2]],gamma=gamma_list[ind[1]],kernel='poly')
model.fit(train_array_X,train_array_Y)
y_predict=model.predict(test_array_X)
score=0
for i in range(0,len(test_array_Y)):
    if test_array_Y[i]==y_predict[i]:
        score+=1
accuracy=(score/len(y_predict))*100
print("Accuracy: ", accuracy)
print("C:", c_list[ind[0]])
print("Gamma:",gamma_list[ind[1]])
print("Degree: ",degree[ind[2]])

Accuracy:  96.22692725298589
C: 1.27
Gamma: 0.1
Degree:  5


In [0]:
total_time


In [0]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
Axes3D.scatter(range(len(total_)), ys, zs=0,